In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
data.head()

In [ ]:
(data.isnull().sum()/data.shape[1])>50

In [ ]:
for i,j in zip(data.columns,data.isnull().sum()/data.shape[1]>50):
    if(j==True):
        print(i," has been dropped")
        data=data.drop(str(i),axis=1)

In [ ]:
#These columns appear to be null
#Later implement the datatype matching using regex because int32,64,128 as well as float stuff
alter_list=[]
for i,j in zip(data.columns,data.isnull().sum()):
    if(j>0):
        if(data[i].dtypes == "float64" or data[i].dtypes == "int64"):
            data_type="Numeric"
        elif(data[i].dtypes == "object"):
            data_type="String"
        else:
            data_type=""
        print(i,j," Data type being : ",data_type)
        alter_list.append((i,data_type))

In [ ]:
# data['Age'] = data['Age'].fillna(data['Age'].mean())
# data['Embarked'] = data['Embarked'].fillna('S')
# data = data.drop(['Name', 'Ticket', 'Fare', 'PassengerId'], axis=1)
for i in alter_list:
    if i[1] == "Numeric":
        data[i[0]] = data[i[0]].fillna(data[i[0]].mean())
    elif i[1]=="String":
        print("Unique values in the column",i[0],data[i[0]].unique())
        print("NUmber of unique vlaues in column",i[0], data[i[0]].nunique())
        print("Number of each unique value occurances made in column",i[0]+":\n",data[i[0]].value_counts(dropna=False))
        import math
        toleration_limit = math.floor(data.shape[0]*0.1)
        if data[i[0]].nunique()>toleration_limit:
            data = data.drop(i[0],axis=1)
        else:
            maxm_occ = data[i[0]].value_counts(dropna=False)[:1].index.tolist()
            print(maxm_occ[0])
            data[i[0]].fillna(maxm_occ[0])
            
#         data[i[0]] = data[i[0]].fillna(data[i[0]])

In [ ]:
data = pd.get_dummies(data)
print(data.head())
X=data.drop("Survived",1)
y=data["Survived"]
data.isnull().sum()

In [ ]:
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

In [ ]:
#Using Pearson Correlation
cor = data.corr()
imp_fact = abs(cor['Survived'])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_fact.plot(kind = "barh")
plt.show()
plt.figure(figsize=(10,10))
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["Survived"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
relevant_features

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
model = LogisticRegression(fit_intercept=True)

In [ ]:
survived_train = data.Survived
data = data.drop("Survived",axis=1)
train_data = data.values[:600]
labels = survived_train[:600]
eval_data = data.values[600:]
eval_labels = survived_train[600:]

In [ ]:
model.fit(train_data, labels)
eval_predictions = model.predict(eval_data)

In [ ]:
print('Accuracy of the model on train data: {0}'.format(model.score(train_data, labels)))

In [ ]:
print('Accuracy of the model on eval data: {0}'.format(model.score(eval_data, eval_labels)))

In [ ]:
data
